In [ ]:
import torch
import os
from pathlib import Path
# change base folder
os.chdir('../')
# Load your model definition and dataset
from models import get_model
from types import SimpleNamespace
import yaml
import matplotlib.pyplot as plt
from flame_model.FLAME import FLAMEModel
from renderer.renderer import Renderer
from pytorch3d.transforms import matrix_to_euler_angles
import matplotlib.animation as animation
import numpy as np
from dataset.data_loader_joint_data_batched import get_dataloaders
from scipy.signal import savgol_filter

device   = torch.device("cuda" if torch.cuda.is_available() else "cpu")
flame    = FLAMEModel(n_shape=300,n_exp=50).to(device)
renderer = Renderer(render_full_head=True).to(device)

In [ ]:
def load_and_flatten_yaml(config_path):
    """
    Loads the YAML file and flattens the structure so that
    all sub-keys under top-level sections (e.g., DATA, NETWORK, etc.)
    appear in a single dictionary without the top-level keys.
    """
    with open(config_path, 'r') as f:
        full_config = yaml.safe_load(f)

    # Flatten the dict by merging all sub-dicts
    flattened_config = {}
    for top_level_key, sub_dict in full_config.items():
        # sub_dict should itself be a dict of key-value pairs
        if isinstance(sub_dict, dict):
            # Merge each sub-key into flattened_config
            for k, v in sub_dict.items():
                flattened_config[k] = v
        else:
            # In case there's a non-dict top-level key (unlikely but possible)
            flattened_config[top_level_key] = sub_dict

    return SimpleNamespace(**flattened_config)

In [ ]:
device   = torch.device("cuda" if torch.cuda.is_available() else "cpu")

global cfg

cfg = load_and_flatten_yaml("config/joint_data/stage1.yaml")

cfg.batch_size = 1

In [ ]:
model = get_model(cfg)
model = model.to(device)
model = model.eval()

In [ ]:
dataset = get_dataloaders(cfg,False)

train_loader = dataset['train']

In [ ]:
num_samples = 1
for i, (blendshapes, mask) in enumerate(train_loader):
    if i >= num_samples:
        break
    blendshapes = blendshapes.to(device)
    mask = mask.to(device)

    with torch.no_grad():
        blendshapes_out, quant_loss = model.forward_time(blendshapes, mask)

In [ ]:
num_samples = 1
for i, (blendshapes, mask) in enumerate(train_loader):
    if i >= num_samples:
        break
    blendshapes = blendshapes.to(device)
    mask = mask.to(device)

    with torch.no_grad():
        blendshapes_out, quant_loss = model(blendshapes, mask)